# リバーシを作る

## 仕様
- 必須機能
  - 以下に示すような画面例を持つリバーシをプレイできる。（リバーシのルールは検索して調べること）
  - 人vs人対戦
  - 入力はキーボードからの数字とアルファべット→input()?
  - ゲーム終了時に勝敗，白と黒の石の数を表示
- 任意機能
  - 人vs ＣＰＵモード、CPU vs CPUモードの追加
  - 対戦結果の外部ファイルへの出力
  - ゲーム途中のセーブ機能とロード機能
  - その他の機能を自由に追加して構わない

## メモ
枠をつくる(8*8)
中央2*2[4,4], [4,5], [5,4], [5,5]の
○●
●○←が必要

白と黒の石（○と●）
挟んだら色が反転する
相手の色を挟める場所しか置けない
挟めなかったらパス
全部埋まるor どっちも置けない→ゲーム終了

### 作る関数
1. 盤面を表示
2. 石を置く
3. 石が置けるかをチェック

1. board関数
   tablesize*tablesizeの配列をつくる（intにする）
   初期条件：[3][3],[4][4]に白，[3][4]，[4][3]に黒
3. display関数
   配列の各要素に白，黒，空白を割り当て8*8できれいに表示
4. check関数
   盤内においたか
   →周りの8マスに相手の石があるか
    ない→faulse
    ある→その直線上の石を調べる
        自分の石→faulse
        相手の石→loop
5. put関数
   置く座標と色を指定
   →board[x][y] = colorとする
            
    

In [1]:
!pip install keyboard

In [7]:
import numpy as np
import sys
import keyboard

white = 1   # ○
black = -1  # ●
blank = 0
tablesize = 8

class Reversi(object):
    
    def __init__(self):
        self.cell = np.zeros((tablesize, tablesize))
        self.cell = self.cell.astype(int)
        self.cell[3][3] = self.cell[4][4] = black
        self.cell[3][4] = self.cell[4][3] = white
        self.current = black
        self.dx = [-1, 0, 1]
        self.dy = [-1, 0, 1]
        self.turn = 1
        
    def _check(self, board, put, flip=False):
        y, x = put
        player = self.current
        if not (0 <= y < tablesize and 0 <= x < tablesize) or board[y][x] != blank:
            return False
        
        found = False
        
        for dy in self.dy:
            for dx in self.dx:
                if dy == 0 and dx == 0:
                    continue
                
                ny, nx = y+dy, x+dx
                flipped = []
                
                while(0 <= ny < tablesize and 0 <= nx < tablesize) and board[ny][nx] == -player:
                    flipped.append((ny, nx))
                    ny += dy
                    nx += dx
                
                if len(flipped) > 0 and (0 <= ny < tablesize and 0 <= nx < tablesize) and board[ny][nx] == player:
                    found = True
                    if flip:
                        board[y][x] = player
                        for fy, fx in flipped:
                            board[fy][fx] = player
        return found
                    
                
    def _display(self):
        print("   0  1  2  3  4  5  6  7")
        for i in range(tablesize):
            print(f"{i}", end="  ")
            for j in range(tablesize):
                if self.cell[i][j] == 0:
                    print("-", end="  ")
                elif self.cell[i][j] == 1:
                    print("○", end="  ")
                else:
                    print("●", end="  ")
            print()
        print("-" * 30)
    
    def _get_move(self):
        valid = []
        for r in range(tablesize):
            for c in range(tablesize):
                put = (r, c)
                if self._check(self.cell, put, flip=False):
                    valid.append(put)
        return valid
    
    def play(self):
        pass_count = 0
        while True:
            self._display()
            valid = self._get_move()
            current_p = self.current
            
            if np.all(self.cell != blank):
                print("全埋まり")
                break
            
            if not valid:
                self.current = -self.current
                opp_valid = self._get_move()
                self.current = -self.current
                if not opp_valid:
                    print("連パス")
                    break
                print(f"{current_p}がパス")
                pass_count += 1
                if pass_count == 2:
                    print("連パス")
                    break
                self.current = -self.current
                continue
            
            while True:
                print(f"↓player{"●" if self.current == black else "○"} 入力待ち")
                user_input = input("(i j): qを押して強制終了").strip().lower()
                if user_input == "q":
                    print("！"*25)
                    print("！"*10 + "ゲーム終了" + "！"*10)
                    print("！"*25)
                    sys.exit()
                try:
                    y, x = map(int, user_input.split())
                    put = (y, x)
                    if put in valid:
                        print(f"player{"●" if self.current == black else "○"} (i j): {put}")
                        break
                    else:
                        print("正しくない")
                except:
                    print("(y x)を入力してください")
            self._check(self.cell, put, flip=True)
            self.current = -self.current
        
        b_score = np.sum(self.cell == black)
        w_score = np.sum(self.cell == white)
        print(f"最終スコア: ●{b_score} - ●{w_score}")

if __name__ == "__main__":
    Reversi().play()

   0  1  2  3  4  5  6  7
0  -  -  -  -  -  -  -  -  
1  -  -  -  -  -  -  -  -  
2  -  -  -  -  -  -  -  -  
3  -  -  -  ○  ●  -  -  -  
4  -  -  -  ●  ○  -  -  -  
5  -  -  -  -  -  -  -  -  
6  -  -  -  -  -  -  -  -  
7  -  -  -  -  -  -  -  -  
------------------------------
↓player● 入力待ち
！！！！！！！！！！！！！！！！！！！！！！！！！
！！！！！！！！！！ゲーム終了！！！！！！！！！！
！！！！！！！！！！！！！！！！！！！！！！！！！


SystemExit: 

/Users/kg/.pyenv/versions/3.12.10/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# import numpy as np

# # 定数
# white = 1
# black = -1 # 手番の初期値と一致
# blank = 0
# tablesize = 8

# class Reversi(object):
    
#     def __init__(self):
#         # 盤面の初期化
#         self.cell = np.zeros((tablesize, tablesize)).astype(int)
#         self.cell[3][3] = self.cell[4][4] = white
#         self.cell[3][4] = self.cell[4][3] = black
        
#         # 現在の手番
#         self.current = black # 黒からスタート
        
#         # 探索方向
#         self.dx = [-1, 0, 1]
#         self.dy = [-1, 0, 1]
        
#     def _check(self, board, put, flip=False):
#         # put は (y, x) のタプルであると想定
#         y, x = put
#         current_player = self.current
        
#         # 1. 初期チェック：範囲外または既に石があるか
#         if not (0 <= y < tablesize and 0 <= x < tablesize) or board[y][x] != blank:
#             return False

#         found_flip = False
        
#         # 2. 8方向 (dy, dx) の探索ループ
#         for dy_val in self.dy:
#             for dx_val in self.dx:
#                 if dx_val == 0 and dy_val == 0:
#                     continue

#                 ny, nx = y + dy_val, x + dx_val
#                 flipped_discs = []

#                 # 3. 相手の石の連続探索
#                 while (0 <= ny < tablesize and 0 <= nx < tablesize) and \
#                       board[ny][nx] == -current_player: # 相手の石
                    
#                     flipped_discs.append((ny, nx))
#                     ny += dy_val
#                     nx += dx_val

#                 # 4. 挟み込みの判定
#                 if len(flipped_discs) > 0 and \
#                    (0 <= ny < tablesize and 0 <= nx < tablesize) and \
#                    board[ny][nx] == current_player: # 終点が自分の石
                    
#                     found_flip = True
                    
#                     # 5. 反転処理 (flip=True の場合のみ実行)
#                     if flip:
#                         # 最初に置いたマスに石を置く
#                         board[y][x] = current_player
                        
#                         # 挟んだ石をすべて反転する
#                         for fy, fx in flipped_discs:
#                             board[fy][fx] = current_player
        
#         return found_flip

#     # --- ShortReversiの表示機能に相当 ---
#     def _display(self):
#         """盤面をコンソールに表示する"""
#         print('  0 1 2 3 4 5 6 7')
#         discs = {white: '⚪', black: '⚫', blank: '・'}
        
#         for r in range(tablesize):
#             row_str = f'{r} '
#             for c in range(tablesize):
#                 row_str += discs[self.cell[r][c]] + ' '
#             print(row_str)
#         #
#         print(f"\nCurrent Turn: {'⚫' if self.current == black else '⚪'}")
#         print(f"Scores: ⚫ {np.sum(self.cell == black)}, ⚪ {np.sum(self.cell == white)}\n")

#     # --- 合法手のリストアップ ---
#     def _get_valid_moves(self):
#         """現在のプレイヤーが石を置けるすべてのマス (y, x) のリストを返す"""
#         valid_moves = []
#         for r in range(tablesize):
#             for c in range(tablesize):
#                 put = (r, c)
#                 # _checkをflip=Falseで呼び出し、合法手であるかを確認
#                 if self._check(self.cell, put, flip=False):
#                     valid_moves.append(put)
#         return valid_moves

#     # --- ShortReversiのplay機能に相当 ---
#     def play(self):
#         """二人対戦のゲームループ"""
#         pass_count = 0
        
#         while True:
#             self._display()
#             valid_moves = self._get_valid_moves()
            
#             # 1. パスの判定
#             if not valid_moves:
#                 print(f"{'⚫' if self.current == black else '⚪'} (Player {self.current}) has no valid moves. PASS.")
#                 pass_count += 1
                
#                 # 2回連続パスでゲーム終了
#                 if pass_count == 2:
#                     print("--- GAME END ---")
#                     break
                
#                 # パスの場合、ターンだけを切り替える
#                 self.current = -self.current
#                 continue 

#             pass_count = 0 # パスでない場合はリセット

#             # 2. プレイヤーの入力
#             while True:
#                 try:
#                     print(f"Player {'⚫' if self.current == black else '⚪'} (Y X): ", end='')
#                     y, x = map(int, input().split())
#                     put = (y, x)
                    
#                     # 3. 入力された手が合法手かチェックし、合法なら処理を抜ける
#                     if put in valid_moves:
#                         break
#                     else:
#                         print("Invalid move. Please choose a valid (Y X) coordinate.")
#                 except:
#                     print("Invalid input format. Please enter two numbers (Y X) separated by space.")

#             # 4. 着手と反転処理
#             self._check(self.cell, put, flip=True)
            
#             # 5. ターンの切り替え (ShortReversiの self.turn = 3 - self.turn に相当)
#             self.current = -self.current 

#         # 6. 最終結果の表示
#         black_score = np.sum(self.cell == black)
#         white_score = np.sum(self.cell == white)
        
#         print(f"Final Score: ⚫ {black_score} vs ⚪ {white_score}")
#         if black_score > white_score:
#             print("Winner: ⚫ Black!")
#         elif white_score > black_score:
#             print("Winner: ⚪ White!")
#         else:
#             print("Draw!")

# if __name__ == '__main__':
#     Reversi().play()

  0 1 2 3 4 5 6 7
0 ・ ・ ・ ・ ・ ・ ・ ・ 
1 ・ ・ ・ ・ ・ ・ ・ ・ 
2 ・ ・ ・ ・ ・ ・ ・ ・ 
3 ・ ・ ・ ⚪ ⚫ ・ ・ ・ 
4 ・ ・ ・ ⚫ ⚪ ・ ・ ・ 
5 ・ ・ ・ ・ ・ ・ ・ ・ 
6 ・ ・ ・ ・ ・ ・ ・ ・ 
7 ・ ・ ・ ・ ・ ・ ・ ・ 

Current Turn: ⚫
Scores: ⚫ 2, ⚪ 2

Player ⚫ (Y X): Invalid input format. Please enter two numbers (Y X) separated by space.
Player ⚫ (Y X): Invalid input format. Please enter two numbers (Y X) separated by space.
Player ⚫ (Y X): Invalid input format. Please enter two numbers (Y X) separated by space.
Player ⚫ (Y X): Invalid input format. Please enter two numbers (Y X) separated by space.
Player ⚫ (Y X): Invalid input format. Please enter two numbers (Y X) separated by space.
Player ⚫ (Y X): Invalid input format. Please enter two numbers (Y X) separated by space.
Player ⚫ (Y X): Invalid input format. Please enter two numbers (Y X) separated by space.
Player ⚫ (Y X): Invalid input format. Please enter two numbers (Y X) separated by space.
Player ⚫ (Y X): 

In [1]:
import numpy as np
import sys
import keyboard

white = 1   # ○
black = -1  # ●
blank = 0
tablesize = 8

class Reversi(object):
    
    def __init__(self):
        self.cell = np.zeros((tablesize, tablesize))
        self.cell = self.cell.astype(int)
        self.cell[3][3] = self.cell[4][4] = white
        self.cell[3][4] = self.cell[4][3] = black
        self.current = black
        self.dx = [-1, 0, 1]
        self.dy = [-1, 0, 1]
        self.turn = 1
        
    def _check(self, board, put, flip=False):
        y, x = put
        player = self.current
        if not (0 <= y < tablesize and 0 <= x < tablesize) or board[y][x] != blank:
            return False
        
        found = False
        
        for dy in self.dy:
            for dx in self.dx:
                if dy == 0 and dx == 0:
                    continue
                
                ny, nx = y+dy, x+dx
                flipped = []
                
                while(0 <= ny < tablesize and 0 <= nx < tablesize) and board[ny][nx] == -player:
                    flipped.append((ny, nx))
                    ny += dy
                    nx += dx
                
                if len(flipped) > 0 and (0 <= ny < tablesize and 0 <= nx < tablesize) and board[ny][nx] == player:
                    found = True
                    if flip:
                        board[y][x] = player
                        for fy, fx in flipped:
                            board[fy][fx] = player
        return found
                    
                
    def _display(self):
        print("-" * 30)
        print("   0  1  2  3  4  5  6  7")
        for i in range(tablesize):
            print(f"{i}", end="  ")
            for j in range(tablesize):
                if self.cell[i][j] == 0:
                    print("-", end="  ")
                elif self.cell[i][j] == 1:
                    print("○", end="  ")
                else:
                    print("●", end="  ")
            print()
        print("-" * 30)
    
    def _get_move(self):
        valid = []
        for r in range(tablesize):
            for c in range(tablesize):
                put = (r, c)
                if self._check(self.cell, put, flip=False):
                    valid.append(put)
        return valid
    
    def _fastestB(self):   # 黒が埋まる
        print("(first:black)\n45 → 55 → 54 → 35 → 24 → 13 → 23 → 53 → 32 → 31")
    def _fastestW(self):   # 白が埋まる
        print("(first:black)\n53 → 34 → 23 → 52 → 45 → 54 → 63 → 44 → 41")
    
    def play(self):
        score_count_b = 0
        score_count_w = 0
        draw_count = 0
        pass_count = 0
        
        while True:
            print("\n===== New Game =====")
            self.__init__()
            while True:
                self._display()
                valid = self._get_move()
                current_p = self.current
                
                if np.all(self.cell != blank):
                    print("The board is full")
                    break
                
                if not valid:
                    self.current = -self.current
                    opp_valid = self._get_move()
                    self.current = -self.current
                    if not opp_valid:
                        print("Skipped: Both has no valid moves")
                        break
                    print(f"Skipped: {current_p} has no valid moves")
                    pass_count += 1
                    if pass_count == 2:
                        print("Skipped Both has no valid moves")
                        break
                    self.current = -self.current
                    continue
                
                while True:
                    print(f"\n↓ Waiting for player{"●" if self.current == black else "○"} input... (ex: 3 4)")
                    user_input = input("press \"q\" to quit the game\n(i j): ").strip().lower()
                    if user_input == "q" or user_input == "Q":
                        print("\n===== Quitted the game =====")
                        sys.exit()
                    elif user_input == "full black":
                        self._fastestB()
                    elif user_input == "full white":
                        self._fastestW()
                    try:
                        y, x = map(int, user_input.split())
                        put = (y, x)
                        if put in valid:
                            print(f"\nplayer{"●" if self.current == black else "○"} (i j): {put}")
                            break
                        else:
                            print("\n!!!!! Cannot put this position !!!!!")
                    except:
                        print("\n!!!!! Invalid input: Enter the (i j) ex: 3 4 !!!!!")
                self._check(self.cell, put, flip=True)
                self.current = -self.current
            
            b_score = np.sum(self.cell == black)
            w_score = np.sum(self.cell == white)
            if b_score > w_score:
                print("● Black Win")
                score_count_b += 1
            elif b_score < w_score:
                print("○ White Win")
                score_count_w += 1
            else:
                print("Draw")
                draw_count += 1
            print(f"Score: ●{b_score} - ○{w_score}")
            print(f"Total score: ●win {score_count_b} - ○win {score_count_w} - draw {draw_count}")
            
            again = input("\n Play again? (y/n)").strip().lower()
            if again != "y":
                print("===== ｵﾜﾀ ====")
                break
        
if __name__ == "__main__":
    Reversi().play()





===== New Game =====
------------------------------
   0  1  2  3  4  5  6  7
0  -  -  -  -  -  -  -  -  
1  -  -  -  -  -  -  -  -  
2  -  -  -  -  -  -  -  -  
3  -  -  -  ○  ●  -  -  -  
4  -  -  -  ●  ○  -  -  -  
5  -  -  -  -  -  -  -  -  
6  -  -  -  -  -  -  -  -  
7  -  -  -  -  -  -  -  -  
------------------------------

↓ Waiting for player● input... (ex: 3 4)

===== Quitted the game =====


SystemExit: 

/Users/kg/.pyenv/versions/3.12.10/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
class ShortReversi:
    def __init__(self):
        self.dirs = [-10, -9, -8, -1, 1, 8, 9, 10]
        self.discs = ' - ○ \n'
        self.board = [0 if i % 9 else 3 for i in range(91)]
        self.board[40] = self.board[50] = self.turn = 1
        self.board[41] = self.board[49] = 2

    def _check(self, board, move, flip=False):
        if not (ret := False) and not board[move]:
            for i in range(8):
                count = value = move + self.dirs[i]
                while board[value] == 3 - self.turn:
                    value += self.dirs[i]
                if count != value and board[value] == self.turn:
                    ret = value = move
                    while flip:
                        board[value] = self.turn
                        value += self.dirs[i]
                        if board[value] == self.turn:
                            break
        return ret

    def play(self, com1=False, com2=True):
        end = False
        while not (move := 0):
            for i in range(9, 82):
                if self._check(self.board, i, flip=False) and not move:
                    move = i
                print(self.discs[self.board[i]*2:][:2], end='')
            while move and not (end := False):
                if not com1 and self.turn == 1 or not com2 and self.turn == 2:
                    x, y = [int(i) for i in input().split()]
                    move = x + y * 9
                if self._check(self.board, move, flip=True):
                    break
            else:
                if end:
                    break
                end, _ = True, print('pass')
            self.turn = 3 - self.turn


if __name__ == '__main__':
    ShortReversi().play()


 - - - - - - - - - - - - - - - - - - - - - - - - - - - ○ 
 - - - - - - 
 ○ - - - - - - - - - - - - - - - - - - - - - - - - - - -

ValueError: not enough values to unpack (expected 2, got 0)